In [21]:
from itertools import chain
from lnn import Predicate, Variable, Exists, Implies, And, ForAll, Model, Fact, World

![Hierarchy](https://upload.wikimedia.org/wikipedia/commons/thumb/3/31/Quadrilateral_hierarchy_svg.svg/629px-Quadrilateral_hierarchy_svg.svg.png)

In [22]:
entities = [
    "3-sides-equal trapezoid",
    "bicentric",
    "complex",
    "concave",
    "convex",
    "cyclic",
    "isosceles trapezoid",
    "kite",
    "parallelogram",
    "quadrilateral",
    "rectangle",
    "rhombus",
    "right-angled trapezoid",
    "simple",
    "square",
    "tangential",
    "trapezoid",
]

entity_relationships = [
    ("square", "rhombus"),
    ("square", "bicentric"),
    ("square", "rectangle"),
    ("square", "3-sides-equal trapezoid"),
    ("rhombus", "kite"),
    ("rhombus", "parallelogram"),
    ("bicentric", "tangential"),
    ("bicentric", "cyclic"),
    ("rectangle", "parallelogram"),
    ("rectangle", "right-angled trapezoid"),
    ("rectangle", "isosceles trapezoid"),
    ("3-sides-equal trapezoid", "isosceles trapezoid"),
    ("kite", "tangential"),
    ("parallelogram", "trapezoid"),
    ("right-angled trapezoid", "trapezoid"),
    ("isosceles trapezoid", "trapezoid"),
    ("tangential", "convex"),
    ("trapezoid", "convex"),
    ("cyclic", "convex"),
    ("convex", "simple"),
    ("concave", "simple"),
    ("complex", "quadrilateral"),
]

for entity1, entity2 in entity_relationships:
    assert entity1 in entities
    assert entity2 in entities

properties = [
    # quadrilateral
    "foursided",
    # simple
    "not-self_intersecting", # negation?
    # complex
    "self-intersecting",
    # convex
    "no-reflex-interior-angle",
    # concave
    "atleast-one-reflex-interior-angle", # negation?
    # tangential
    "sides-are-tangents-to-inscribed-circle",
    # trapezoid
    "atleast-one-pair-opposite-sides-parallel",
    # cyclic
    "four-verticles-on-circumscribed-circle",
    # kite
    "two-pairs-adjacent-sides-equal-length",
    # parallelogram
    "two-pairs-opposite-sides-parallel",
    # right angled trapezoid
    "two-adjacent-right-angles",
    # isosceles trapezoid
    "base-angles-are-same",
    # rhombus
    "all-sides-equal-length",
    # rectangle
    "all-right-angles",
    # 3-sides-equal trapezoid
    "atleast-three-congruent-sides"
]

entity_properties = [
    ("quadrilateral", "foursided"),
    ("simple", "not-self_intersecting"),
    ("complex", "self-intersecting"),
    ("convex", "no-reflex-interior-angle"),
    ("concave", "atleast-one-reflex-interior-angle"),
    ("tangential", "sides-are-tangents-to-inscribed-circle"),
    ("trapezoid", "atleast-one-pair-opposite-sides-parallel"),
    ("cyclic", "four-verticles-on-circumscribed-circle"),
    ("kite", "two-pairs-adjacent-sides-equal-length"),
    ("parallelogram", "two-pairs-opposite-sides-parallel"),
    ("right-angled trapezoid", "two-adjacent-right-angles"),
    ("isosceles trapezoid", "base-angles-are-same"),
    ("rhombus", "all-sides-equal-length"),
    ("rectangle", "all-right-angles"),
    ("3-sides-equal trapezoid", "atleast-three-congruent-sides")
]

for entity, property in entity_properties:
    assert entity in entities
    assert property in properties


In [23]:
# gegnerate one object of all entity types
facts = dict([(entity, [f"{entity}1"]) for entity in entities])
facts

{'3-sides-equal trapezoid': ['3-sides-equal trapezoid1'],
 'bicentric': ['bicentric1'],
 'complex': ['complex1'],
 'concave': ['concave1'],
 'convex': ['convex1'],
 'cyclic': ['cyclic1'],
 'isosceles trapezoid': ['isosceles trapezoid1'],
 'kite': ['kite1'],
 'parallelogram': ['parallelogram1'],
 'quadrilateral': ['quadrilateral1'],
 'rectangle': ['rectangle1'],
 'rhombus': ['rhombus1'],
 'right-angled trapezoid': ['right-angled trapezoid1'],
 'simple': ['simple1'],
 'square': ['square1'],
 'tangential': ['tangential1'],
 'trapezoid': ['trapezoid1']}

In [24]:
model = Model()

x = Variable("x")

# parsing predicates
for predicate in chain(entities, properties):
    model[predicate] = Predicate(name=predicate)

In [25]:
# parsing axioms in Implication and ForAll shape
axioms = [
    ForAll(
        x,
        Implies(model[p1](x), model[p2](x), name=f"{p1}_{p2}"),
        name=f"all-{p1}_{p2}",
        world=World.AXIOM,
    )
    for p1, p2 in chain(entity_relationships, entity_properties)
]

model.add_formulae(*axioms)

In [26]:
# parsing facts
model.add_facts(
    {entity: {name: Fact.TRUE for name in names} for entity, names in facts.items()}
)

In [27]:
queries = [
    Exists(x, model["foursided"](x), name="foursided_objects"),
    Exists(
        x,
        And(model["all-right-angles"](x), model["all-sides-equal-length"](x)),
        name="all-right-angles-and-all-sides-equal-length",
    )
]

model.add_formulae(*queries)

In [28]:
steps, facts_inferred = model.infer()

for query in queries:
    print(f"Query {query.name} result:")
    print(model[query.name].true_groundings)

Query foursided_objects result:
{'quadrilateral1', 'complex1'}
Query all-right-angles-and-all-sides-equal-length result:
{'square1'}


In [29]:
model.print()


***************************************************************************
                                LNN Model

OPEN   Exists: all-right-angles-and-all-sides-equal-length (x)          TRUE (1.0, 1.0)

OPEN   And: And_2(x) 
'parallelogram1'                                         UNKNOWN (0.0, 1.0)
'rhombus1'                                               UNKNOWN (0.0, 1.0)
'rectangle1'                                             UNKNOWN (0.0, 1.0)
'isosceles trapezoid1'                                   UNKNOWN (0.0, 1.0)
'square1'                                                   TRUE (1.0, 1.0)
'kite1'                                                  UNKNOWN (0.0, 1.0)
'right-angled trapezoid1'                                UNKNOWN (0.0, 1.0)
'bicentric1'                                             UNKNOWN (0.0, 1.0)
'trapezoid1'                                             UNKNOWN (0.0, 1.0)
'simple1'                                                UNKNOWN (0.0, 1.0)
'concave1